In [60]:
import matplotlib as plt
import numpy as np
import pandas as pd
import math
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data] Error downloading 'stopwords' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/corpora/stopwords.zip>:   <urlopen
[nltk_data]     error [Errno 11004] getaddrinfo failed>


False

In [64]:
#Resources for text cleaning: 
#https://machinelearningmastery.com/clean-text-machine-learning-python/
#https://github.com/SudalaiRajkumar/NLP/blob/master/src/Text%20Cleaning.ipynb

#Sentence tokenization
from nltk import sent_tokenize
def getSentTokens(sentences):
    return sent_tokenize(sentences)

#Word tokenization
from nltk import word_tokenize
def getWordTokens(sentence):
    return word_tokenize(sentence)

#Punctuations removal
def removePunctuations(word):
    return re.sub(r'\W+', '', word)

#Stop words removal from list of words
from nltk.corpus import stopwords
eng_stop=set(stopwords.words('english'))
print(eng_stop)
def removeStopwords(text):
    return [word for word in text if word not in eng_stop]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Searched in:
    - 'C:\\Users\\Lenovo/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Lenovo\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Lenovo\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Lenovo\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data'
**********************************************************************


In [47]:
text = "This's is@ 9 short, sentence. Amazing!"

In [48]:
sentences=getSentTokens(text)
print(sentences[0])
print(sentences[1])

This's is@ 9 short, sentence.
Amazing!


In [65]:
tokens=getWordTokens(sentences[1])
print(tokens)
tokens=getWordTokens(sentences[0])
print(tokens)

['Amazing', '!']
['This', "'s", 'is', '@', '9', 'short', ',', 'sentence', '.']


In [66]:
#Convert everything into lowercase
tokens=[w.lower() for w in tokens]
print(tokens)

['this', "'s", 'is', '@', '9', 'short', ',', 'sentence', '.']


In [67]:
tokens=[removePunctuations(w) for w in tokens]
tokens=list( filter(None,tokens) )
print(tokens)
print(len(tokens))

['this', 's', 'is', '9', 'short', 'sentence']
6


In [ ]:
tokens=removeStopwords(tokens)
print(tokens)
print(len(tokens))